In [7]:
import requests
import arrow
import pprint
import json
from urllib.parse import urlencode
from functools import reduce

In [2]:
token = "secret_XEztYu6AfhifDHSrTyKtmbFoaUkU2rjiXCcYO7nXp04"

In [3]:
extra_data = {"filter": {"and": [{"property": "标签",
                                  "multi_select": {"is_not_empty": True}},],},}

In [4]:
r_database = requests.post(
    url="https://api.notion.com/v1/databases/cecf4bb039dc46bca130a29a9db58906/query",
    headers={"Authorization": "Bearer " + token,
             "Notion-Version": "2021-08-02",
             "Content-Type": "application/json",
             },
    data=json.dumps(extra_data),
)

In [5]:
respond = json.loads(r_database.text)

In [129]:
#pprint.pprint(respond["results"][-1])

In [27]:
def take_page_plain_text(respond: dict):
    for result in respond["results"]:
        page_id = result["url"].split("/")[-1].split("-")[-1]
        r_page = requests.get(
                    url=f"https://api.notion.com/v1/blocks/{page_id}/children",
                    headers={"Authorization": f"Bearer {token}",
                             "Notion-Version": "2021-08-02",
                             "Content-Type": "application/json",
                             },
                    )
        if not json.loads(r_page.text).get("results", []):
            print(json.loads(r_page.text))
            print("=" * 10)
        for block in json.loads(r_page.text).get("results", []):
#             if not block.get("paragraph"):
#                 print(block)
#                 print("-" * 10)
            for key in block:
                if not isinstance(block[key], dict):
                    continue
                if "text" not in block[key]:
                    continue
                for text in block[key]["text"]:
                    yield text["plain_text"]

In [29]:
text_list = list(take_page_plain_text(respond))

{'object': 'list', 'results': [], 'next_cursor': None, 'has_more': False}


In [30]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba

In [31]:
split_text_list = [jieba.lcut(text, HMM=True) for text in text_list]

In [32]:
split_text_list

[['效果',
  '无法',
  '评估',
  '和',
  '不可',
  '证伪',
  '才',
  '是',
  '这',
  '类产品',
  '的',
  '神秘',
  '所在',
  '。',
  '”',
  '在',
  '陈洁',
  '看来',
  '，',
  '很多',
  '所谓',
  '的',
  '星座',
  '博主',
  '、',
  '占卜师',
  '，',
  '不过',
  '是',
  '利用',
  '了',
  '心理学',
  '上',
  '的',
  '“',
  '巴纳',
  '姆',
  '效应',
  '”',
  '—',
  '—',
  '人',
  '很',
  '容易',
  '相信',
  '一个',
  '笼统',
  '的',
  '一般性',
  '的',
  '人格',
  '描述',
  '，',
  '会',
  '通过',
  '自我',
  '代入',
  '和',
  '内容',
  '延伸',
  '来',
  '证明',
  '它',
  '是',
  '对',
  '的',
  '。'],
 ['事实上',
  '，',
  '星座',
  '博主',
  '的',
  '粉丝',
  '们',
  '，',
  '都',
  '存在',
  '“',
  '幸存者',
  '偏差',
  '”',
  '。',
  '也就是说',
  '，',
  '愿意',
  '发声',
  '的',
  '是',
  '那些',
  '觉得',
  '星座',
  '很准',
  '的',
  '人',
  '，',
  '久而久之',
  '，',
  '信息',
  '市场',
  '上',
  '流通',
  '的',
  '都',
  '是',
  '“',
  '星座',
  '很准',
  '”',
  '的',
  '声音',
  '，',
  '制造',
  '出',
  '一种',
  '大众',
  '都',
  '对此',
  '表示',
  '认同',
  '的',
  '“',
  '假象',
  '”',
  '。'],
 ['而',
  '很多',
  '星座',
  '博主',
  '，',
  '就',
  '利用',
  '了',